In [8]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['문장 감정 정도 예측', '대화 주제 식별', 'LM(한국어)']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            recent_speaker = None
            domain = line['ConversationInfo']
            new_conversation = []
            for utt in line['Conversation']:
                speaker = utt['SpeakerNo']
                if recent_speaker != speaker:
                    new_conversation.append(
                        {   'text_id' : utt['TextNo'],
                            'speaker': speaker, 'text': utt['Text'].replace('  ',' ').strip(),
                            'emotion': 
                                {'category': utt['VerifyEmotionCategory'], 
                                'state': utt['VerifyEmotionTarget'],
                                'intensity': utt['VerifyEmotionLevel']}})
                    recent_speaker = speaker
                else:
                    new_conversation[-1]['text'] += ' ' + utt['Text'].replace('  ',' ').strip()
            #### data preprocess end 

            #### 문장 감정 정도 예측
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            data['input']['info'] = {
                'domain': domain,
                'Speaker1': line['Speaker1'],
                'Speaker2': line['Speaker2'],
            }
            data['input']['dialog'] = [{key:utt[key] for key in ['text_id', 'speaker', 'text']} for utt in new_conversation]
            data['output'] = {'emotions' : [{key:utt[key] for key in ['text_id', 'speaker', 'emotion']} for utt in new_conversation]}
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 문장 감정 정도 예측 end


            #### 대화 주제 식별
            # data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input']['info'].pop('domain')
            data['output'] = domain
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 대화 주제 식별 end


            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = '## domain : ' + str(domain) + '\n## dialog : \n'
            for utt in new_conversation:
                text += f"{utt['speaker']} : {utt['text']}\n"
            data['text'] = text
            ## preprocess data from line end
            task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 2/2 [00:02<00:00,  1.41s/it]
